<a href="https://colab.research.google.com/github/ratnesh003/Learning-NLP/blob/main/Assignment%207/nlp_ui21cs48_lab7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')
nltk.download('treebank')
nltk.download('universal_tagset')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

In [ ]:
data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [ ]:
random.seed(1234)
train_set, test_set = train_test_split(data, train_size=0.95, test_size=0.05)

print("Training Set Length -", len(train_set))
print("Testing Set Length -", len(test_set))
print("-" * 100)
print("Training Data -\n")
print(train_set[:10])

Training Set Length - 3718
Testing Set Length - 196
----------------------------------------------------------------------------------------------------
Training Data -

[[('Pro-forma', 'ADJ'), ('balance', 'NOUN'), ('sheets', 'NOUN'), ('clearly', 'ADV'), ('show', 'VERB'), ('why', 'ADV'), ('Cray', 'NOUN'), ('Research', 'NOUN'), ('favored', 'VERB'), ('the', 'DET'), ('spinoff', 'NOUN'), ('*T*-1', 'X'), ('.', '.')], [('The', 'DET'), ('minimum-wage', 'NOUN'), ('bill', 'NOUN'), ('worked', 'VERB'), ('out', 'PRT'), ('*', 'X'), ('by', 'ADP'), ('Congress', 'NOUN'), ('and', 'CONJ'), ('Bush', 'NOUN'), ('won', 'VERB'), ('easy', 'ADJ'), ('approval', 'NOUN'), ('in', 'ADP'), ('the', 'DET'), ('House', 'NOUN'), ('.', '.')], [('Mr.', 'NOUN'), ('Klauser', 'NOUN'), ('says', 'VERB'), ('0', 'X'), ('Mitsui', 'NOUN'), ('has', 'VERB'), ('75', 'NUM'), ('U.S.', 'NOUN'), ('subsidiaries', 'NOUN'), ('in', 'ADP'), ('which', 'DET'), ('it', 'PRON'), ('holds', 'VERB'), ('35', 'NUM'), ('%', 'NOUN'), ('interest', 'NOUN'),

In [ ]:
train_tagged_words = [tup for sent in train_set for tup in sent]
print("Train Tagged Words - ", len(train_tagged_words))

test_tagged_words = [tup[0] for sent in test_set for tup in sent]
print("Train Tagged Words - ", len(test_tagged_words))

Train Tagged Words -  95418
Train Tagged Words -  5258


In [ ]:
train_tagged_tokens = [tag[0] for tag in train_tagged_words]
train_tagged_tokens[:10]

['Pro-forma',
 'balance',
 'sheets',
 'clearly',
 'show',
 'why',
 'Cray',
 'Research',
 'favored',
 'the']

In [ ]:
train_tagged_pos_tokens = [tag[1] for tag in train_tagged_words]
train_tagged_pos_tokens[:10]

['ADJ', 'NOUN', 'NOUN', 'ADV', 'VERB', 'ADV', 'NOUN', 'NOUN', 'VERB', 'DET']

In [ ]:
training_vocabulary_set = set(train_tagged_tokens)


In [ ]:
training_pos_tag_set = set(train_tagged_pos_tokens)

In [ ]:
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1] == tag]
    tag_count = len(tag_list)
    word_given_tag_list = [pair[0] for pair in tag_list if pair[0] == word]
    word_given_tag_count = len(word_given_tag_list)

    return (word_given_tag_count, tag_count)

In [ ]:
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]

    t1_tags_list = [tag for tag in tags if tag == t1]
    t1_tags_count = len(t1_tags_list)

    t2_given_t1_list = [tags[index+1] for index in range(len(tags)-1) if tags[index] == t1 and tags[index+1] == t2]
    t2_given_t1_count = len(t2_given_t1_list)

    return(t2_given_t1_count, t1_tags_count)

In [ ]:
len_pos_tags = len(training_pos_tag_set)
len_vocab = len(training_vocabulary_set)

tags_matrix = np.zeros((len_pos_tags, len_pos_tags), dtype='float32')
for i, t1 in enumerate(list(training_pos_tag_set)):
    for j, t2 in enumerate(list(training_pos_tag_set)):
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [ ]:
tags_df = pd.DataFrame(tags_matrix, columns = list(training_pos_tag_set), index=list(training_pos_tag_set))

In [ ]:
def Vanilla_Viterbi(words, train_bag = train_tagged_words):
    state = []

    T = list(set([pair[1] for pair in train_bag]))

    for key, word in enumerate(words):
        p = []
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]

            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p
            p.append(state_probability)

        pmax = max(p)
        state_max = T[p.index(pmax)]
        state.append(state_max)
    return list(zip(words, state))

In [ ]:
sentence_test_1 = 'I want to play football'
words = word_tokenize(sentence_test_1)
tagged_seq = Vanilla_Viterbi(words)
print(tagged_seq)

[('I', 'PRON'), ('want', 'VERB'), ('to', 'PRT'), ('play', 'VERB'), ('football', 'NOUN')]


In [ ]:
sentence_test_2='The children played games in the park'
words = word_tokenize(sentence_test_2)
tagged_seq = Vanilla_Viterbi(words)
print(tagged_seq)

[('The', 'DET'), ('children', 'NOUN'), ('played', 'VERB'), ('games', 'NOUN'), ('in', 'ADP'), ('the', 'DET'), ('park', 'NOUN')]


In [ ]:
sentence_test_2='I was busy talking to others.'
words = word_tokenize(sentence_test_2)
tagged_seq = Vanilla_Viterbi(words)
print(tagged_seq)

[('I', 'PRON'), ('was', 'VERB'), ('busy', 'ADJ'), ('talking', 'VERB'), ('to', 'PRT'), ('others', 'NOUN'), ('.', '.')]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

file_path = '/content/drive/MyDrive/ML/spam.csv'
dataset = pd.read_csv(file_path,encoding='latin-1')

In [ ]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt_tab')
predicted_tags_list = []
size = len(dataset['v2'])

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
predicted_tags_list = []
size = len(dataset['v2'])
print(size)

5572


In [ ]:
print(dataset['v2'])

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568              Will ï¿½_ b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: v2, Length: 5572, dtype: object


In [ ]:
sentence_test_2= dataset['v2'][0]
words = word_tokenize(sentence_test_2)
tagged_seq = Vanilla_Viterbi(words)
print(tagged_seq)

[('Go', 'ADJ'), ('until', 'ADP'), ('jurong', 'ADJ'), ('point', 'NOUN'), (',', '.'), ('crazy', 'ADJ'), ('..', 'ADJ'), ('Available', 'ADJ'), ('only', 'ADV'), ('in', 'ADP'), ('bugis', 'ADJ'), ('n', 'ADJ'), ('great', 'ADJ'), ('world', 'NOUN'), ('la', 'DET'), ('e', 'ADJ'), ('buffet', 'NOUN'), ('...', '.'), ('Cine', 'ADJ'), ('there', 'DET'), ('got', 'VERB'), ('amore', 'ADJ'), ('wat', 'ADJ'), ('...', '.')]
